## Car Accidents During COVID Stay-At-Home Periods Analysis
This notebook is using data collected and cleaned in the `car_accidents_eda.ipynb` notebook, and will be used for examining differences state-by-state for severity and frequency of car accidents.  Tableau graphs will be used for final graphs, but this notebook gives a sense of direction.

### TABLE OF CONTENTS
1. [Setup](#Setup)
2. [Top 20 States Analysis](#Top-20-States-Analysis)
    - [California](#California)
    - [Texas](#Texas)
    - [Florida](#Florida)
    - [South Carolina](#South-Carolina)
    - [North Carolina](#North-Carolina)
    - [New York](#New-York)
    - [Pennsylvania](#Pennsylvania)
    - [Illinois](#Illinois)
    - [Virginia](#Virginia)
    - [Michigan](#Michigan)
    - [Georgia](#Georgia)
    - [Oregon](#Oregon)
    - [Minnesota](#Minnesota)
    - [Arizona](#Arizona)
    - [Tennessee](#Tennessee)
    - [Washington](#Washington)
    - [Ohio](#Ohio)
    - [Louisiana](#Louisiana)
    - [Oklahoma](#Oklahoma)
    - [New Jersey](#New-Jersey)
3. [Top 20 States Averages](#Top-20-States-Averages)

First, import libraries and modules needed >

In [1]:
import pickle
import pandas as pd
import numpy as np

### Setup

First import the dataset created and prepared for analysis >

In [2]:
# open our master dataset for analysis

with open('pickle/car_accidents_master.pickle','rb') as read_file:
    car_accidents_master = pickle.load(read_file)

Note that we are looking at our dataset before it was narrowed for modeling (e.g. only severity 2 & 3, only using mapquest-sourced data).  For the analysis we'll be doing here we can look at the full dataset.

But first we'll narrow to only the columns we'll be using, which are date, state, whether in period of shutdown, and severity of the accident >

In [3]:
accident_covid = car_accidents_master[['Severity','Date',
                                       'State','Year','Month',
                                       'Day','Day_Of_Week',
                                       'Shut_Down'
                                      ]]

Now we'll look at amount of accidents in each states >

In [4]:
accident_covid.State.value_counts()

CA    816790
TX    329264
FL    258000
SC    173255
NC    165956
NY    160803
PA    106748
IL     99686
VA     96069
MI     95959
GA     93611
OR     90104
MN     81700
AZ     78584
TN     69895
WA     68541
OH     66137
LA     61512
OK     59972
NJ     59052
MD     53588
UT     51672
CO     49721
AL     44619
MA     39037
IN     33746
MO     33638
CT     25898
NE     23967
KY     22553
WI     20108
RI     11753
IA     11457
NV     10634
NH      7981
KS      7937
MS      6585
DE      5739
NM      5523
DC      4818
WV      2381
ME      2242
ID      2044
AR      2012
VT       701
MT       511
WY       507
SD        61
ND        44
Name: State, dtype: int64

And then see if we narrowed the number of states, how many we'd need to get to at least 75% of total accidents >

In [5]:
accident_covid.State.value_counts()[:20].sum()/accident_covid.State.value_counts().sum()

0.8629486936806794

In [6]:
accident_covid.State.value_counts()[:20]

CA    816790
TX    329264
FL    258000
SC    173255
NC    165956
NY    160803
PA    106748
IL     99686
VA     96069
MI     95959
GA     93611
OR     90104
MN     81700
AZ     78584
TN     69895
WA     68541
OH     66137
LA     61512
OK     59972
NJ     59052
Name: State, dtype: int64

First we'll focus on the state with the highest amount of accidents, as they will provided more data to then draw better conclusions from and also are of greater interest since more need to address car accidents.  We'll focus on the 20 states that have the most car accidents, which also represent 86% of total accidents in our full dataset for the US.  Those states are: CA, TX, FL, SC, NC, NY, PA, IL, VA, MI, GA, OR, MN, AZ, TN, WA, OH, LA, OK, and NJ.  These states also vary in time period of shutdown and political leanings that may influence behavior around a shutdown period, so we have nice diversity for comparison.

For each state, we'll create dataframes for separate periods we want to analyze and compare.  We'll compare stay-at-home order periods with past three years in that same period.

First we'll create functions to get the data we'll need for each state >

In [7]:
def severity_percent(state_data):
    '''
    Takes in state data and returns
    a list of percentages for each 
    severity level to total accidents.
    '''
    percentages = []
    if 1 not in sorted(state_data.Severity.unique()):
        percentages.append(0)
    
    for x in sorted(state_data.Severity.unique()):
        percent = state_data.Severity.value_counts()[x]/state_data.Severity.count()
        percentages.append(round((percent*100),2))
    
    if 4 not in sorted(state_data.Severity.unique()):
        percentages.append(0)
    
    return percentages

In [8]:
def change_freq(state_freq_data):
    '''
    Takes in the dataframe for fequency of 
    car accidents for a state and outputs
    the percent change from stay-home 2020
    period compared to average of equivalent
    periods 2017-2019 rounded to 2 decimals.
    '''
    freq_2020 = state_freq_data.Num_Accidents[1]
    avg_freq = np.mean(state_freq_data.Num_Accidents[2:5])
    
    return round(((freq_2020 - avg_freq)/avg_freq * 100),2)

-
## Top 20 States Analysis

### California

First create periods we'll want to compare for the state >

In [11]:
# full state period
ca_ttl = accident_covid[(accident_covid.State == 'CA')]

# shutdown period
ca_sd = accident_covid[
    (accident_covid.State == 'CA') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
# note CA is still in shutdown, so we're comparing to when dataset ended in 2020
#    not when their stay-at-home orders ended

ca_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-19') & 
                         (car_accidents_master['Date'] <= '2019-06-30') & 
                         (car_accidents_master['State'] == 'CA')]

# 2018
ca_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-19') & 
                         (car_accidents_master['Date'] <= '2018-06-30') & 
                         (car_accidents_master['State'] == 'CA')]

# 2017
ca_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-19') & 
                         (car_accidents_master['Date'] <= '2017-06-30') & 
                         (car_accidents_master['State'] == 'CA')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [12]:
# find total frequencies for state total and in each period 

ca_freqs = [ca_ttl.shape[0],
            ca_sd.shape[0],
            ca_2019.shape[0],
            ca_2018.shape[0],
            ca_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
ca_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': ca_freqs})

ca_freq_data

,Timeframe,Num_Accidents
0,Total,816790
1,Shut_Down,76131
2,2019,38440
3,2018,46203
4,2017,44799


In [13]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(ca_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 76.44%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [14]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

ca_sp_ttl = severity_percent(ca_ttl)
ca_sp_sd = severity_percent(ca_sd)
ca_sp_2019 = severity_percent(ca_2019)
ca_sp_2018 = severity_percent(ca_2018)
ca_sp_2017 = severity_percent(ca_2017)

# then again create a simple dataframe for comparison

ca_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': ca_sp_ttl,
                                 'Shut_Down': ca_sp_sd,
                                 '2019': ca_sp_2019,
                                 '2018': ca_sp_2018,
                                 '2017': ca_sp_2017,
                                })

ca_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,0.71,7.20,0.01,0.02,0.06
1,2,70.61,80.00,64.69,60.23,62.30
2,3,27.65,10.54,33.61,38.90,36.52
3,4,1.04,2.27,1.69,0.85,1.11


#### CA Takeaways:
- See increase in fequency during shut down period compared to same periods in past three years
- Seeing increase in 'edges' of severity -- 1 & 4 -- of car accidents with lowering of 'middle' severities -- 2 & 3 -- for shutdown period compared to average ttl and past year's prior periods

-
### Texas

First create periods we'll want to compare for the state >

In [15]:
# full state period
tx_ttl = accident_covid[(accident_covid.State == 'TX')]

# shutdown period
tx_sd = accident_covid[
    (accident_covid.State == 'TX') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
tx_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-31') & 
                         (car_accidents_master['Date'] <= '2019-04-30') & 
                         (car_accidents_master['State'] == 'TX')]

# 2018
tx_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-31') & 
                         (car_accidents_master['Date'] <= '2018-04-30') & 
                         (car_accidents_master['State'] == 'TX')]

# 2017
tx_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-31') & 
                         (car_accidents_master['Date'] <= '2017-04-30') & 
                         (car_accidents_master['State'] == 'TX')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [16]:
# find total frequencies for state total and in each period 

tx_freqs = [tx_ttl.shape[0],
            tx_sd.shape[0],
            tx_2019.shape[0],
            tx_2018.shape[0],
            tx_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
tx_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': tx_freqs})

tx_freq_data

,Timeframe,Num_Accidents
0,Total,329264
1,Shut_Down,4391
2,2019,6496
3,2018,6823
4,2017,7809


In [17]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(tx_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: -37.65%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [18]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

tx_sp_ttl = severity_percent(tx_ttl)
tx_sp_sd = severity_percent(tx_sd)
tx_sp_2019 = severity_percent(tx_2019)
tx_sp_2018 = severity_percent(tx_2018)
tx_sp_2017 = severity_percent(tx_2017)

# then again create a simple dataframe for comparison

tx_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': tx_sp_ttl,
                                 'Shut_Down': tx_sp_sd,
                                 '2019': tx_sp_2019,
                                 '2018': tx_sp_2018,
                                 '2017': tx_sp_2017
                                })

tx_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,0.32,6.40,0.03,0.03,0.03
1,2,71.01,54.18,72.52,70.22,73.52
2,3,27.23,34.50,25.94,28.64,25.27
3,4,1.43,4.92,1.51,1.11,1.19


#### TX Takeaways:
- Frequency: Unlike CA, TX had a drop in frequency during shutdown compared to past periods
- Severity: Similar to CA, still the 'edges' of severity (1 & 4) growing, 4 in particular to a greater degree than CA 

-
### Florida

First create periods we'll want to compare for the state >

In [19]:
# full state period
fl_ttl = accident_covid[(accident_covid.State == 'FL')]

# shutdown period
fl_sd = accident_covid[
    (accident_covid.State == 'FL') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
fl_2019 = accident_covid[(car_accidents_master['Date'] > '2019-04-03') & 
                         (car_accidents_master['Date'] <= '2019-04-30') & 
                         (car_accidents_master['State'] == 'FL')]

# 2018
fl_2018 = accident_covid[(car_accidents_master['Date'] > '2018-04-03') & 
                         (car_accidents_master['Date'] <= '2018-04-30') & 
                         (car_accidents_master['State'] == 'FL')]

# 2017
fl_2017 = accident_covid[(car_accidents_master['Date'] > '2017-04-03') & 
                         (car_accidents_master['Date'] <= '2017-04-30') & 
                         (car_accidents_master['State'] == 'FL')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [20]:
# find total frequencies for state total and in each period 

fl_freqs = [fl_ttl.shape[0],
            fl_sd.shape[0],
            fl_2019.shape[0],
            fl_2018.shape[0],
            fl_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
fl_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': fl_freqs})

fl_freq_data

,Timeframe,Num_Accidents
0,Total,258000
1,Shut_Down,4941
2,2019,4623
3,2018,5023
4,2017,4256


In [21]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(fl_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 6.62%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [22]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

fl_sp_ttl = severity_percent(fl_ttl)
fl_sp_sd = severity_percent(fl_sd)
fl_sp_2019 = severity_percent(fl_2019)
fl_sp_2018 = severity_percent(fl_2018)
fl_sp_2017 = severity_percent(fl_2017)

# then again create a simple dataframe for comparison

fl_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': fl_sp_ttl,
                                 'Shut_Down': fl_sp_sd,
                                 '2019': fl_sp_2019,
                                 '2018': fl_sp_2018,
                                 '2017': fl_sp_2017
                                })

fl_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,1.17,17.63,0.00,0.00,0.07
1,2,64.15,58.23,65.35,62.45,70.32
2,3,31.23,20.50,29.12,35.24,27.68
3,4,3.46,3.64,5.54,2.31,1.93


#### FL Takeaways:
- Frequency: Not really much change positively or negatively for shutdown compared to past periods
- Severity: Don't see a lot of difference in severity 4 as with CA & TX, but do see significant increase in 1s while 2 & 3 dropped

-
### South Carolina

First create periods we'll want to compare for the state >

In [23]:
# full state period
sc_ttl = accident_covid[(accident_covid.State == 'SC')]

# shutdown period
sc_sd = accident_covid[
    (accident_covid.State == 'SC') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
sc_2019 = accident_covid[(car_accidents_master['Date'] > '2019-04-06') & 
                         (car_accidents_master['Date'] <= '2019-05-04') & 
                         (car_accidents_master['State'] == 'SC')]

# 2018
sc_2018 = accident_covid[(car_accidents_master['Date'] > '2018-04-06') & 
                         (car_accidents_master['Date'] <= '2018-05-04') & 
                         (car_accidents_master['State'] == 'SC')]

# 2017
sc_2017 = accident_covid[(car_accidents_master['Date'] > '2017-04-06') & 
                         (car_accidents_master['Date'] <= '2017-05-04') & 
                         (car_accidents_master['State'] == 'SC')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [24]:
# find total frequencies for state total and in each period 

sc_freqs = [sc_ttl.shape[0],
            sc_sd.shape[0],
            sc_2019.shape[0],
            sc_2018.shape[0],
            sc_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
sc_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': sc_freqs})

sc_freq_data

,Timeframe,Num_Accidents
0,Total,173255
1,Shut_Down,4142
2,2019,4481
3,2018,5119
4,2017,455


In [25]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(sc_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 23.58%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [26]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

sc_sp_ttl = severity_percent(sc_ttl)
sc_sp_sd = severity_percent(sc_sd)
sc_sp_2019 = severity_percent(sc_2019)
sc_sp_2018 = severity_percent(sc_2018)
sc_sp_2017 = severity_percent(sc_2017)

# then again create a simple dataframe for comparison

sc_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': sc_sp_ttl,
                                 'Shut_Down': sc_sp_sd,
                                 '2019': sc_sp_2019,
                                 '2018': sc_sp_2018,
                                 '2017': sc_sp_2017
                                })

sc_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,0.07,0.56,0.02,0.02,0.22
1,2,79.28,90.56,77.15,76.40,81.98
2,3,19.98,8.16,21.85,23.36,14.95
3,4,0.68,0.72,0.98,0.21,2.86


#### SC Takeaways:
- Frequency: no significant difference for shutdown period.  2017 seems low, but this may be due to less data collected that year. 
- Severity: in general, seemed to lower in severity.  See more 1 & 2 and less 3 & 4

-
### North Carolina

First create periods we'll want to compare for the state >

In [27]:
# full state period
nc_ttl = accident_covid[(accident_covid.State == 'NC')]

# shutdown period
nc_sd = accident_covid[
    (accident_covid.State == 'NC') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
nc_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-30') & 
                         (car_accidents_master['Date'] <= '2019-05-22') & 
                         (car_accidents_master['State'] == 'NC')]

# 2018
nc_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-30') & 
                         (car_accidents_master['Date'] <= '2018-05-22') & 
                         (car_accidents_master['State'] == 'NC')]

# 2017
nc_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-30') & 
                         (car_accidents_master['Date'] <= '2017-05-22') & 
                         (car_accidents_master['State'] == 'NC')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [28]:
# find total frequencies for state total and in each period 

nc_freqs = [nc_ttl.shape[0],
            nc_sd.shape[0],
            nc_2019.shape[0],
            nc_2018.shape[0],
            nc_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
nc_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': nc_freqs})

nc_freq_data

,Timeframe,Num_Accidents
0,Total,165956
1,Shut_Down,7641
2,2019,7539
3,2018,9659
4,2017,1530


In [29]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(nc_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 22.40%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [30]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

nc_sp_ttl = severity_percent(nc_ttl)
nc_sp_sd = severity_percent(nc_sd)
nc_sp_2019 = severity_percent(nc_2019)
nc_sp_2018 = severity_percent(nc_2018)
nc_sp_2017 = severity_percent(nc_2017)

# then again create a simple dataframe for comparison

nc_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': nc_sp_ttl,
                                 'Shut_Down': nc_sp_sd,
                                 '2019': nc_sp_2019,
                                 '2018': nc_sp_2018,
                                 '2017': nc_sp_2017
                                })

nc_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,1.09,8.17,0.04,0.03,0.00
1,2,83.79,75.71,85.30,85.11,91.18
2,3,13.28,12.75,12.20,13.72,4.18
3,4,1.84,3.38,2.45,1.14,4.64


#### NC Takeaways:
- Fequency: No significant change during shutdown period
- Severity: Do see an increase in level 4 severity, but greatest change is increase in severity 1 with drop in severity 2

-
### New York

First create periods we'll want to compare for the state >

In [97]:
# full state period
ny_ttl = accident_covid[(accident_covid.State == 'NY')]

# shutdown period
ny_sd = accident_covid[
    (accident_covid.State == 'NY') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
ny_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-22') & 
                         (car_accidents_master['Date'] <= '2019-06-13') & 
                         (car_accidents_master['State'] == 'NY')]

# 2018
ny_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-22') & 
                         (car_accidents_master['Date'] <= '2018-06-13') & 
                         (car_accidents_master['State'] == 'NY')]

# 2017
ny_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-22') & 
                         (car_accidents_master['Date'] <= '2017-06-13') & 
                         (car_accidents_master['State'] == 'NY')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [98]:
# find total frequencies for state total and in each period 

ny_freqs = [ny_ttl.shape[0],
            ny_sd.shape[0],
            ny_2019.shape[0],
            ny_2018.shape[0],
            ny_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
ny_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': ny_freqs})

ny_freq_data

,Timeframe,Num_Accidents
0,Total,160803
1,Shut_Down,12442
2,2019,10586
3,2018,7933
4,2017,6319


In [99]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(ny_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 50.28%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [100]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

ny_sp_ttl = severity_percent(ny_ttl)
ny_sp_sd = severity_percent(ny_sd)
ny_sp_2019 = severity_percent(ny_2019)
ny_sp_2018 = severity_percent(ny_2018)
ny_sp_2017 = severity_percent(ny_2017)

# then again create a simple dataframe for comparison

ny_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': ny_sp_ttl,
                                 'Shut_Down': ny_sp_sd,
                                 '2019': ny_sp_2019,
                                 '2018': ny_sp_2018,
                                 '2017': ny_sp_2017
                                })

ny_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,0.45,4.55,0.02,0.04,0.03
1,2,59.73,55.33,66.75,54.00,55.15
2,3,35.37,31.43,30.35,42.28,39.42
3,4,4.44,8.70,2.88,3.68,5.40


#### NY Takeaways:
- Frequency: See an increase in shutdown period, although there was also an even larger jump from 2018 to 2019
- Severity: See increase in edge severities (1 & 2)

-
### Pennsylvania

First create periods we'll want to compare for the state >

In [101]:
# full state period
pa_ttl = accident_covid[(accident_covid.State == 'PA')]

# shutdown period
pa_sd = accident_covid[
    (accident_covid.State == 'PA') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
pa_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-23') & 
                         (car_accidents_master['Date'] <= '2019-06-04') & 
                         (car_accidents_master['State'] == 'PA')]

# 2018
pa_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-23') & 
                         (car_accidents_master['Date'] <= '2018-06-04') & 
                         (car_accidents_master['State'] == 'PA')]

# 2017
pa_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-23') & 
                         (car_accidents_master['Date'] <= '2017-06-04') & 
                         (car_accidents_master['State'] == 'PA')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [102]:
# find total frequencies for state total and in each period 

pa_freqs = [pa_ttl.shape[0],
            pa_sd.shape[0],
            pa_2019.shape[0],
            pa_2018.shape[0],
            pa_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
pa_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': pa_freqs})

pa_freq_data

,Timeframe,Num_Accidents
0,Total,106748
1,Shut_Down,8576
2,2019,3633
3,2018,4344
4,2017,5510


In [103]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(pa_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 90.76%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [104]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

pa_sp_ttl = severity_percent(pa_ttl)
pa_sp_sd = severity_percent(pa_sd)
pa_sp_2019 = severity_percent(pa_2019)
pa_sp_2018 = severity_percent(pa_2018)
pa_sp_2017 = severity_percent(pa_2017)

# then again create a simple dataframe for comparison

pa_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': pa_sp_ttl,
                                 'Shut_Down': pa_sp_sd,
                                 '2019': pa_sp_2019,
                                 '2018': pa_sp_2018,
                                 '2017': pa_sp_2017
                                })

pa_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,0.21,1.59,0.00,0.05,0.05
1,2,73.73,65.64,67.96,67.31,80.34
2,3,20.72,26.38,26.26,28.13,13.03
3,4,5.35,6.40,5.78,4.51,6.57


#### PA Takeaways:
- Frequency: See a fairly significant increase in accidents during shutdown period
- Severity: See increase in severity 1 and drop in severity 2, but not that large

-
### Illinois

First create periods we'll want to compare for the state >

In [39]:
# full state period
il_ttl = accident_covid[(accident_covid.State == 'IL')]

# shutdown period
il_sd = accident_covid[
    (accident_covid.State == 'IL') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
il_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-25') & 
                         (car_accidents_master['Date'] <= '2019-05-31') & 
                         (car_accidents_master['State'] == 'IL')]

# 2018
il_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-25') & 
                         (car_accidents_master['Date'] <= '2018-05-31') & 
                         (car_accidents_master['State'] == 'IL')]

# 2017
il_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-25') & 
                         (car_accidents_master['Date'] <= '2017-05-31') & 
                         (car_accidents_master['State'] == 'IL')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [40]:
# find total frequencies for state total and in each period 

il_freqs = [il_ttl.shape[0],
            il_sd.shape[0],
            il_2019.shape[0],
            il_2018.shape[0],
            il_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
il_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': il_freqs})

il_freq_data

,Timeframe,Num_Accidents
0,Total,99686
1,Shut_Down,5037
2,2019,3681
3,2018,3714
4,2017,4737


In [41]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(il_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 24.55%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [42]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

il_sp_ttl = severity_percent(il_ttl)
il_sp_sd = severity_percent(il_sd)
il_sp_2019 = severity_percent(il_2019)
il_sp_2018 = severity_percent(il_2018)
il_sp_2017 = severity_percent(il_2017)

# then again create a simple dataframe for comparison

il_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': il_sp_ttl,
                                 'Shut_Down': il_sp_sd,
                                 '2019': il_sp_2019,
                                 '2018': il_sp_2018,
                                 '2017': il_sp_2017
                                })

il_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,0.27,3.59,0.00,0.11,0.06
1,2,63.60,18.23,62.70,62.74,69.07
2,3,32.75,72.76,30.64,33.23,27.93
3,4,3.38,5.42,6.66,3.93,2.93


#### IL Takeaways:
- Frequency: Increase during shutdown period
- Severity: Large increase in severity 3 and decrese in severity 2.  Also increase in severity 1

-
### Virginia

First create periods we'll want to compare for the state >

In [43]:
# full state period
va_ttl = accident_covid[(accident_covid.State == 'VA')]

# shutdown period
va_sd = accident_covid[
    (accident_covid.State == 'VA') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
va_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-30') & 
                         (car_accidents_master['Date'] <= '2019-06-10') & 
                         (car_accidents_master['State'] == 'VA')]

# 2018
va_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-30') & 
                         (car_accidents_master['Date'] <= '2018-06-10') & 
                         (car_accidents_master['State'] == 'VA')]

# 2017
va_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-30') & 
                         (car_accidents_master['Date'] <= '2017-06-10') & 
                         (car_accidents_master['State'] == 'VA')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [44]:
# find total frequencies for state total and in each period 

va_freqs = [va_ttl.shape[0],
            va_sd.shape[0],
            va_2019.shape[0],
            va_2018.shape[0],
            va_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
va_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': va_freqs})

va_freq_data

,Timeframe,Num_Accidents
0,Total,96069
1,Shut_Down,8207
2,2019,5175
3,2018,4123
4,2017,2255


In [45]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(va_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 113.11%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [46]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

va_sp_ttl = severity_percent(va_ttl)
va_sp_sd = severity_percent(va_sd)
va_sp_2019 = severity_percent(va_2019)
va_sp_2018 = severity_percent(va_2018)
va_sp_2017 = severity_percent(va_2017)

# then again create a simple dataframe for comparison

va_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': va_sp_ttl,
                                 'Shut_Down': va_sp_sd,
                                 '2019': va_sp_2019,
                                 '2018': va_sp_2018,
                                 '2017': va_sp_2017
                                })

va_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,1.81,15.36,0.04,0.00,0.00
1,2,53.75,58.60,58.28,43.39,49.53
2,3,38.70,20.32,35.83,51.52,41.55
3,4,5.74,5.71,5.86,5.09,8.91


#### VA Takeaways:
- Frequency:  See a large increase in frequency during shutdown
- Severity: Significant increase in severity 1 and drop in severity 3

-
### Michigan

First create periods we'll want to compare for the state >

In [47]:
# full state period
mi_ttl = accident_covid[(accident_covid.State == 'MI')]

# shutdown period
mi_sd = accident_covid[
    (accident_covid.State == 'MI') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
mi_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-24') & 
                         (car_accidents_master['Date'] <= '2019-05-28') & 
                         (car_accidents_master['State'] == 'MI')]

# 2018
mi_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-24') & 
                         (car_accidents_master['Date'] <= '2018-05-28') & 
                         (car_accidents_master['State'] == 'MI')]

# 2017
mi_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-24') & 
                         (car_accidents_master['Date'] <= '2017-05-28') & 
                         (car_accidents_master['State'] == 'MI')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [48]:
# find total frequencies for state total and in each period 

mi_freqs = [mi_ttl.shape[0],
            mi_sd.shape[0],
            mi_2019.shape[0],
            mi_2018.shape[0],
            mi_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
mi_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': mi_freqs})

mi_freq_data

,Timeframe,Num_Accidents
0,Total,95959
1,Shut_Down,2010
2,2019,3392
3,2018,4558
4,2017,4514


In [49]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(mi_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: -51.62%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [50]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

mi_sp_ttl = severity_percent(mi_ttl)
mi_sp_sd = severity_percent(mi_sd)
mi_sp_2019 = severity_percent(mi_2019)
mi_sp_2018 = severity_percent(mi_2018)
mi_sp_2017 = severity_percent(mi_2017)

# then again create a simple dataframe for comparison

mi_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': mi_sp_ttl,
                                 'Shut_Down': mi_sp_sd,
                                 '2019': mi_sp_2019,
                                 '2018': mi_sp_2018,
                                 '2017': mi_sp_2017
                                })

mi_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,0.06,0.95,0.00,0.02,0.00
1,2,59.45,61.44,61.56,59.04,56.76
2,3,34.95,28.31,31.93,35.65,38.83
3,4,5.55,9.30,6.52,5.29,4.41


#### MI Takeaways:
- Frequency: Drop in number of accidents
- Severity: Increase in edges of severity 1 & 4 -- most dramatic with severity 4

-
### Georgia

First create periods we'll want to compare for the state >

In [51]:
# full state period
ga_ttl = accident_covid[(accident_covid.State == 'GA')]

# shutdown period
ga_sd = accident_covid[
    (accident_covid.State == 'GA') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
ga_2019 = accident_covid[(car_accidents_master['Date'] > '2019-04-03') & 
                         (car_accidents_master['Date'] <= '2019-04-30') & 
                         (car_accidents_master['State'] == 'GA')]

# 2018
ga_2018 = accident_covid[(car_accidents_master['Date'] > '2018-04-03') & 
                         (car_accidents_master['Date'] <= '2018-04-30') & 
                         (car_accidents_master['State'] == 'GA')]

# 2017
ga_2017 = accident_covid[(car_accidents_master['Date'] > '2017-04-03') & 
                         (car_accidents_master['Date'] <= '2017-04-30') & 
                         (car_accidents_master['State'] == 'GA')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [52]:
# find total frequencies for state total and in each period 

ga_freqs = [ga_ttl.shape[0],
            ga_sd.shape[0],
            ga_2019.shape[0],
            ga_2018.shape[0],
            ga_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
ga_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': ga_freqs})

ga_freq_data

,Timeframe,Num_Accidents
0,Total,93611
1,Shut_Down,1469
2,2019,1930
3,2018,1755
4,2017,1422


In [53]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(ga_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: -13.71%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [54]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

ga_sp_ttl = severity_percent(ga_ttl)
ga_sp_sd = severity_percent(ga_sd)
ga_sp_2019 = severity_percent(ga_2019)
ga_sp_2018 = severity_percent(ga_2018)
ga_sp_2017 = severity_percent(ga_2017)

# then again create a simple dataframe for comparison

ga_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': ga_sp_ttl,
                                 'Shut_Down': ga_sp_sd,
                                 '2019': ga_sp_2019,
                                 '2018': ga_sp_2018,
                                 '2017': ga_sp_2017
                                })

ga_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,0.43,7.49,0.00,0.00,0.00
1,2,41.58,47.45,45.70,39.03,32.42
2,3,50.03,37.58,45.91,54.02,59.92
3,4,7.96,7.49,8.39,6.95,7.67


-
### Oregon

First create periods we'll want to compare for the state >

In [55]:
# full state period
or_ttl = accident_covid[(accident_covid.State == 'OR')]

# shutdown period
or_sd = accident_covid[
    (accident_covid.State == 'OR') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
or_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-23') & 
                         (car_accidents_master['Date'] <= '2019-06-30') & 
                         (car_accidents_master['State'] == 'OR')]

# 2018
or_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-23') & 
                         (car_accidents_master['Date'] <= '2018-06-30') & 
                         (car_accidents_master['State'] == 'OR')]

# 2017
or_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-23') & 
                         (car_accidents_master['Date'] <= '2017-06-30') & 
                         (car_accidents_master['State'] == 'OR')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >m

In [56]:
# find total frequencies for state total and in each period 

or_freqs = [or_ttl.shape[0],
            or_sd.shape[0],
            or_2019.shape[0],
            or_2018.shape[0],
            or_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
or_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': or_freqs})

or_freq_data

,Timeframe,Num_Accidents
0,Total,90104
1,Shut_Down,9853
2,2019,7688
3,2018,2174
4,2017,433


In [57]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(or_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 187.12%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [58]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

or_sp_ttl = severity_percent(or_ttl)
or_sp_sd = severity_percent(or_sd)
or_sp_2019 = severity_percent(or_2019)
or_sp_2018 = severity_percent(or_2018)
or_sp_2017 = severity_percent(or_2017)

# then again create a simple dataframe for comparison

or_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': or_sp_ttl,
                                 'Shut_Down': or_sp_sd,
                                 '2019': or_sp_2019,
                                 '2018': or_sp_2018,
                                 '2017': or_sp_2017
                                })

or_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,1.40,12.29,0.03,0.00,0.00
1,2,86.27,77.07,89.93,63.29,55.20
2,3,8.96,5.99,7.19,31.83,10.39
3,4,3.37,4.65,2.85,4.88,34.41


-
### Minnesota

First create periods we'll want to compare for the state >

In [59]:
# full state period
mn_ttl = accident_covid[(accident_covid.State == 'MN')]

# shutdown period
mn_sd = accident_covid[
    (accident_covid.State == 'MN') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
mn_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-27') & 
                         (car_accidents_master['Date'] <= '2019-05-17') & 
                         (car_accidents_master['State'] == 'MN')]

# 2018
mn_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-27') & 
                         (car_accidents_master['Date'] <= '2018-05-17') & 
                         (car_accidents_master['State'] == 'MN')]

# 2017
mn_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-27') & 
                         (car_accidents_master['Date'] <= '2017-05-17') & 
                         (car_accidents_master['State'] == 'MN')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [60]:
# find total frequencies for state total and in each period 

mn_freqs = [mn_ttl.shape[0],
            mn_sd.shape[0],
            mn_2019.shape[0],
            mn_2018.shape[0],
            mn_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
mn_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': mn_freqs})

mn_freq_data

,Timeframe,Num_Accidents
0,Total,81700
1,Shut_Down,3354
2,2019,2313
3,2018,2645
4,2017,535


In [61]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(mn_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 83.18%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [62]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

mn_sp_ttl = severity_percent(mn_ttl)
mn_sp_sd = severity_percent(mn_sd)
mn_sp_2019 = severity_percent(mn_2019)
mn_sp_2018 = severity_percent(mn_2018)
mn_sp_2017 = severity_percent(mn_2017)

# then again create a simple dataframe for comparison

mn_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': mn_sp_ttl,
                                 'Shut_Down': mn_sp_sd,
                                 '2019': mn_sp_2019,
                                 '2018': mn_sp_2018,
                                 '2017': mn_sp_2017
                                })

mn_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,0.05,0.36,0.00,0.11,0.00
1,2,65.35,77.61,61.05,38.26,87.85
2,3,34.03,21.14,37.74,61.17,10.65
3,4,0.57,0.89,1.21,0.45,1.50


-
### Arizona

First create periods we'll want to compare for the state >

In [63]:
# full state period
az_ttl = accident_covid[(accident_covid.State == 'AZ')]

# shutdown period
az_sd = accident_covid[
    (accident_covid.State == 'AZ') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
az_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-31') & 
                         (car_accidents_master['Date'] <= '2019-05-15') & 
                         (car_accidents_master['State'] == 'AZ')]

# 2018
az_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-31') & 
                         (car_accidents_master['Date'] <= '2018-05-15') & 
                         (car_accidents_master['State'] == 'AZ')]

# 2017
az_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-31') & 
                         (car_accidents_master['Date'] <= '2017-05-15') & 
                         (car_accidents_master['State'] == 'AZ')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [64]:
# find total frequencies for state total and in each period 

az_freqs = [az_ttl.shape[0],
            az_sd.shape[0],
            az_2019.shape[0],
            az_2018.shape[0],
            az_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
az_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': az_freqs})

az_freq_data

,Timeframe,Num_Accidents
0,Total,78584
1,Shut_Down,4919
2,2019,2777
3,2018,2610
4,2017,469


In [65]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(az_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 152.00%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [66]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

az_sp_ttl = severity_percent(az_ttl)
az_sp_sd = severity_percent(az_sd)
az_sp_2019 = severity_percent(az_2019)
az_sp_2018 = severity_percent(az_2018)
az_sp_2017 = severity_percent(az_2017)

# then again create a simple dataframe for comparison

az_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': az_sp_ttl,
                                 'Shut_Down': az_sp_sd,
                                 '2019': az_sp_2019,
                                 '2018': az_sp_2018,
                                 '2017': az_sp_2017
                                })

az_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,8.53,59.85,0.11,0.00,0.00
1,2,70.10,32.71,77.75,70.27,62.47
2,3,16.77,3.80,18.15,25.06,14.50
3,4,4.60,3.64,4.00,4.67,23.03


-
### Tennessee

First create periods we'll want to compare for the state >

In [67]:
# full state period
tn_ttl = accident_covid[(accident_covid.State == 'TN')]

# shutdown period
tn_sd = accident_covid[
    (accident_covid.State == 'TN') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
tn_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-31') & 
                         (car_accidents_master['Date'] <= '2019-04-30') & 
                         (car_accidents_master['State'] == 'TN')]

# 2018
tn_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-31') & 
                         (car_accidents_master['Date'] <= '2018-04-30') & 
                         (car_accidents_master['State'] == 'TN')]

# 2017
tn_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-31') & 
                         (car_accidents_master['Date'] <= '2017-04-30') & 
                         (car_accidents_master['State'] == 'TN')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [68]:
# find total frequencies for state total and in each period 

tn_freqs = [tn_ttl.shape[0],
            tn_sd.shape[0],
            tn_2019.shape[0],
            tn_2018.shape[0],
            tn_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
tn_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': tn_freqs})

tn_freq_data

,Timeframe,Num_Accidents
0,Total,69895
1,Shut_Down,1614
2,2019,2324
3,2018,2124
4,2017,274


In [69]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(tn_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 2.54%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [70]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

tn_sp_ttl = severity_percent(tn_ttl)
tn_sp_sd = severity_percent(tn_sd)
tn_sp_2019 = severity_percent(tn_2019)
tn_sp_2018 = severity_percent(tn_2018)
tn_sp_2017 = severity_percent(tn_2017)

# then again create a simple dataframe for comparison

tn_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': tn_sp_ttl,
                                 'Shut_Down': tn_sp_sd,
                                 '2019': tn_sp_2019,
                                 '2018': tn_sp_2018,
                                 '2017': tn_sp_2017
                                })

tn_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,2.08,25.28,0.04,0.09,0.00
1,2,65.73,54.89,73.28,58.05,68.61
2,3,29.77,17.91,25.13,39.97,14.60
3,4,2.42,1.92,1.55,1.88,16.79


-
### Washington

First create periods we'll want to compare for the state >

In [71]:
# full state period
wa_ttl = accident_covid[(accident_covid.State == 'WA')]

# shutdown period
wa_sd = accident_covid[
    (accident_covid.State == 'WA') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
wa_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-23') & 
                         (car_accidents_master['Date'] <= '2019-05-31') & 
                         (car_accidents_master['State'] == 'WA')]

# 2018
wa_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-23') & 
                         (car_accidents_master['Date'] <= '2018-05-31') & 
                         (car_accidents_master['State'] == 'WA')]

# 2017
wa_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-23') & 
                         (car_accidents_master['Date'] <= '2017-05-31') & 
                         (car_accidents_master['State'] == 'WA')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [72]:
# find total frequencies for state total and in each period 

wa_freqs = [wa_ttl.shape[0],
            wa_sd.shape[0],
            wa_2019.shape[0],
            wa_2018.shape[0],
            wa_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
wa_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': wa_freqs})

wa_freq_data

,Timeframe,Num_Accidents
0,Total,68541
1,Shut_Down,2461
2,2019,2647
3,2018,3627
4,2017,2572


In [73]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(wa_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: -16.54%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [74]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

wa_sp_ttl = severity_percent(wa_ttl)
wa_sp_sd = severity_percent(wa_sd)
wa_sp_2019 = severity_percent(wa_2019)
wa_sp_2018 = severity_percent(wa_2018)
wa_sp_2017 = severity_percent(wa_2017)

# then again create a simple dataframe for comparison

wa_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': wa_sp_ttl,
                                 'Shut_Down': wa_sp_sd,
                                 '2019': wa_sp_2019,
                                 '2018': wa_sp_2018,
                                 '2017': wa_sp_2017
                                })

wa_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,0.77,13.45,0.00,0.03,0.08
1,2,60.88,62.29,56.14,61.04,59.84
2,3,33.90,19.42,37.21,35.18,36.00
3,4,4.44,4.84,6.65,3.75,4.08


-
### Ohio

First create periods we'll want to compare for the state >

In [75]:
# full state period
oh_ttl = accident_covid[(accident_covid.State == 'OH')]

# shutdown period
oh_sd = accident_covid[
    (accident_covid.State == 'OH') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
oh_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-22') & 
                         (car_accidents_master['Date'] <= '2019-06-13') & 
                         (car_accidents_master['State'] == 'OH')]

# 2018
oh_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-22') & 
                         (car_accidents_master['Date'] <= '2018-06-13') & 
                         (car_accidents_master['State'] == 'OH')]

# 2017
oh_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-22') & 
                         (car_accidents_master['Date'] <= '2017-06-13') & 
                         (car_accidents_master['State'] == 'OH')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [76]:
# find total frequencies for state total and in each period 

oh_freqs = [oh_ttl.shape[0],
            oh_sd.shape[0],
            oh_2019.shape[0],
            oh_2018.shape[0],
            oh_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
oh_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': oh_freqs})

oh_freq_data

,Timeframe,Num_Accidents
0,Total,66137
1,Shut_Down,5805
2,2019,2594
3,2018,3188
4,2017,2211


In [77]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(oh_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 117.88%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [78]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

oh_sp_ttl = severity_percent(oh_ttl)
oh_sp_sd = severity_percent(oh_sd)
oh_sp_2019 = severity_percent(oh_2019)
oh_sp_2018 = severity_percent(oh_2018)
oh_sp_2017 = severity_percent(oh_2017)

# then again create a simple dataframe for comparison

oh_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': oh_sp_ttl,
                                 'Shut_Down': oh_sp_sd,
                                 '2019': oh_sp_2019,
                                 '2018': oh_sp_2018,
                                 '2017': oh_sp_2017
                                })

oh_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,0.80,6.61,0.00,0.03,0.00
1,2,62.17,73.21,59.56,52.63,58.48
2,3,28.58,12.49,34.58,36.89,26.37
3,4,8.45,7.68,5.86,10.45,15.15


-
### Louisiana

First create periods we'll want to compare for the state >

In [79]:
# full state period
la_ttl = accident_covid[(accident_covid.State == 'LA')]

# shutdown period
la_sd = accident_covid[
    (accident_covid.State == 'LA') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
la_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-22') & 
                         (car_accidents_master['Date'] <= '2019-05-15') & 
                         (car_accidents_master['State'] == 'LA')]

# 2018
la_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-22') & 
                         (car_accidents_master['Date'] <= '2018-05-15') & 
                         (car_accidents_master['State'] == 'LA')]

# 2017
la_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-22') & 
                         (car_accidents_master['Date'] <= '2017-05-15') & 
                         (car_accidents_master['State'] == 'LA')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [80]:
# find total frequencies for state total and in each period 

la_freqs = [la_ttl.shape[0],
            la_sd.shape[0],
            la_2019.shape[0],
            la_2018.shape[0],
            la_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
la_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': la_freqs})

la_freq_data

,Timeframe,Num_Accidents
0,Total,61512
1,Shut_Down,2683
2,2019,2946
3,2018,3938
4,2017,211


In [81]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(la_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 13.45%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [82]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

la_sp_ttl = severity_percent(la_ttl)
la_sp_sd = severity_percent(la_sd)
la_sp_2019 = severity_percent(la_2019)
la_sp_2018 = severity_percent(la_2018)
la_sp_2017 = severity_percent(la_2017)

# then again create a simple dataframe for comparison

la_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': la_sp_ttl,
                                 'Shut_Down': la_sp_sd,
                                 '2019': la_sp_2019,
                                 '2018': la_sp_2018,
                                 '2017': la_sp_2017
                                })

la_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,2.05,21.21,0.10,0.10,0.00
1,2,76.57,49.24,78.17,81.82,65.88
2,3,19.38,24.67,19.79,16.66,9.48
3,4,2.00,4.88,1.93,1.42,24.64


-
### Oklahoma

First create periods we'll want to compare for the state >

In [83]:
# full state period
ok_ttl = accident_covid[(accident_covid.State == 'OK')]

# shutdown period
ok_sd = accident_covid[
    (accident_covid.State == 'OK') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
ok_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-24') & 
                         (car_accidents_master['Date'] <= '2019-05-06') & 
                         (car_accidents_master['State'] == 'OK')]

# 2018
ok_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-24') & 
                         (car_accidents_master['Date'] <= '2018-05-06') & 
                         (car_accidents_master['State'] == 'OK')]

# 2017
ok_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-24') & 
                         (car_accidents_master['Date'] <= '2017-05-06') & 
                         (car_accidents_master['State'] == 'OK')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [84]:
# find total frequencies for state total and in each period 

ok_freqs = [ok_ttl.shape[0],
            ok_sd.shape[0],
            ok_2019.shape[0],
            ok_2018.shape[0],
            ok_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
ok_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': ok_freqs})

ok_freq_data

,Timeframe,Num_Accidents
0,Total,59972
1,Shut_Down,1960
2,2019,2938
3,2018,2560
4,2017,136


In [85]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(ok_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 4.37%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [86]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

ok_sp_ttl = severity_percent(ok_ttl)
ok_sp_sd = severity_percent(ok_sd)
ok_sp_2019 = severity_percent(ok_2019)
ok_sp_2018 = severity_percent(ok_2018)
ok_sp_2017 = severity_percent(ok_2017)

# then again create a simple dataframe for comparison

ok_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': ok_sp_ttl,
                                 'Shut_Down': ok_sp_sd,
                                 '2019': ok_sp_2019,
                                 '2018': ok_sp_2018,
                                 '2017': ok_sp_2017
                                })

ok_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,0.12,0.36,0.03,0.04,0.00
1,2,89.32,83.88,92.41,90.20,78.68
2,3,9.82,15.36,7.35,9.34,5.88
3,4,0.73,0.41,0.20,0.43,15.44


-
### New Jersey

First create periods we'll want to compare for the state >

In [87]:
# full state period
nj_ttl = accident_covid[(accident_covid.State == 'NJ')]

# shutdown period
nj_sd = accident_covid[
    (accident_covid.State == 'NJ') & 
    (accident_covid.Shut_Down == 1)]

# 2019 
nj_2019 = accident_covid[(car_accidents_master['Date'] > '2019-03-21') & 
                         (car_accidents_master['Date'] <= '2019-06-30') & 
                         (car_accidents_master['State'] == 'NJ')]

# 2018
nj_2018 = accident_covid[(car_accidents_master['Date'] > '2018-03-21') & 
                         (car_accidents_master['Date'] <= '2018-06-30') & 
                         (car_accidents_master['State'] == 'NJ')]

# 2017
nj_2017 = accident_covid[(car_accidents_master['Date'] > '2017-03-21') & 
                         (car_accidents_master['Date'] <= '2017-06-30') & 
                         (car_accidents_master['State'] == 'NJ')]

#### Frequency
Now compare frequencies (total number) of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018, & 2017 being in same month-day date range as shut-down period) >

In [88]:
# find total frequencies for state total and in each period 

nj_freqs = [nj_ttl.shape[0],
            nj_sd.shape[0],
            nj_2019.shape[0],
            nj_2018.shape[0],
            nj_2017.shape[0]
           ]

# then develop a simple dataframe to compare them
nj_freq_data = pd.DataFrame({'Timeframe': ['Total','Shut_Down',2019,2018,2017],
                                 'Num_Accidents': nj_freqs})

nj_freq_data

,Timeframe,Num_Accidents
0,Total,59052
1,Shut_Down,4724
2,2019,3932
3,2018,4361
4,2017,3642


In [89]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(nj_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 18.74%


#### Severity
Now compare severities of car accidents for total, shut-down period in 2020, 2019, 2018, and 2017 (2019, 2018 & 2017 being in same month-day date range as shut-down period) >

In [90]:
# first find percentages of severity ranks to total accidents for 
#   total and for each period 

nj_sp_ttl = severity_percent(nj_ttl)
nj_sp_sd = severity_percent(nj_sd)
nj_sp_2019 = severity_percent(nj_2019)
nj_sp_2018 = severity_percent(nj_2018)
nj_sp_2017 = severity_percent(nj_2017)

# then again create a simple dataframe for comparison

nj_severity_data = pd.DataFrame({'Severity_Rank': [1,2,3,4],
                                 'TTL': nj_sp_ttl,
                                 'Shut_Down': nj_sp_sd,
                                 '2019': nj_sp_2019,
                                 '2018': nj_sp_2018,
                                 '2017': nj_sp_2017
                                })

nj_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,1,0.16,1.44,0.08,0.05,0.05
1,2,66.31,48.65,68.79,62.88,68.62
2,3,27.16,32.66,25.38,32.22,26.39
3,4,6.38,17.25,5.75,4.86,4.94


-
### Top 20 States Averages

Combining numbers for the Top 20 states to see average trends for those states >

#### Frequency

In [91]:
# combine fequency data for the top 20 states 

top20_freq_data = (ca_freq_data + 
                   tx_freq_data +
                   fl_freq_data +
                   sc_freq_data +
                   nc_freq_data +
                   ny_freq_data +
                   pa_freq_data +
                   il_freq_data +
                   va_freq_data +
                   mi_freq_data +
                   ga_freq_data + 
                   or_freq_data +
                   mn_freq_data +
                   az_freq_data +
                   tn_freq_data +
                   wa_freq_data +
                   oh_freq_data +
                   la_freq_data +
                   ok_freq_data +
                   nj_freq_data)

top20_freq_data['Timeframe'] = ['Total','Shut_Down',2019,2018,2017]

top20_freq_data

,Timeframe,Num_Accidents
0,Total,3031638
1,Shut_Down,172360
2,2019,120135
3,2018,126481
4,2017,94089


Now find change in 2020 shutdown period compared to average of past three years >

In [92]:
# percent change in frequency in 2020 stay-home compared to average past years

change = change_freq(top20_freq_data)
print("Change Frequency: %.2f%%" % (change))

Change Frequency: 51.77%


#### Severity

For average severity rank for top 10 states, we will include weighting for each state based on the states number of accidents compared to all accidents for the top 10 states

In [93]:
# finding weight for each state to then include in next cell

len(accident_covid[accident_covid.State == 'NJ'])/accident_covid.State.value_counts()[:20].sum()

0.019478578906848378

In [94]:
# add all state severity dfs together

top20_severity_data = (ca_severity_data * 27 +
                       tx_severity_data * 11 +
                       fl_severity_data * 9 +
                       sc_severity_data * 6 +
                       nc_severity_data * 5 +
                       ny_severity_data * 5 +
                       pa_severity_data * 4 +
                       il_severity_data * 3 +
                       va_severity_data * 3 +
                       mi_severity_data * 3 +
                       ga_severity_data * 3 +
                       or_severity_data * 3 +
                       mn_severity_data * 3 +
                       az_severity_data * 3 +
                       tn_severity_data * 2 +
                       wa_severity_data * 2 +
                       oh_severity_data * 2 +
                       la_severity_data * 2 +
                       ok_severity_data * 2 +
                       nj_severity_data * 2 )

# get average for the 20 states
top20_severity_data = round(top20_severity_data.div(104),2)

top20_severity_data

,Severity_Rank,TTL,Shut_Down,2019,2018,2017
0,0.96,0.88,8.97,0.02,0.03,0.05
1,1.92,66.49,63.51,65.91,60.82,64.51
2,2.88,26.14,19.60,27.16,33.01,26.17
3,3.85,2.64,4.07,3.06,2.30,5.43


In [95]:
# (2020 value - average 2017-2019) / average 2017-2019 * 100

sev1_2020 = top20_severity_data.Shut_Down[0]

sev1_avg_past = np.mean(top20_severity_data['2017'][0] + 
                        top20_severity_data['2018'][0] + 
                        top20_severity_data['2019'][0])

print('Percent Change in 2020 stay-home period vs. average 2017-2019:')
((sev1_2020 - sev1_avg_past) / sev1_avg_past) * 100

Percent Change in 2020 stay-home period vs. average 2017-2019:


8870.0

#### Top 20 Takeaways:
- Frequency: See a 51% increase during shutdown period compared to past periods
- Severity: See greatest shift in severity 1 -- an increase of 8,870%.  Severity 4's are also showing an increase compared to 2019 and 2018, but not to 2017.  Changes in severity 4 seem to be more on a state-by-state basis